In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(data.table)
library(readxl)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: data.table 1.12.0  Latest news: r-datatable.com

  warnings.warn(x, RRuntimeWarning)


In [272]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bsp
import time
import re

In [14]:
## test
i=0
parameters={"area":"china",
           "type":"MovieRankingHistory",
           "category":"all",
           "page":f'{i}',
           "display":"list",
           "timestamp":f'{time.time()}',
           "version":"07bb781100018dd58eafc3b35d42686804c6df8d",
           "dataType":"json"}
MtimeRank = rq.get("http://movie.mtime.com/boxoffice/",params=parameters)
print(MtimeRank.status_code)
MtimeRank.text

200


'{"expire":"2019-02-11T03:48:59.935Z","html":"<div class=\\"bocontent\\"><div><div class=\\"typetitle\\"><h4>内地影史票房</h4><p class=\\"updateday\\">更新时间：2019年2月11日</p><div class=\\"showtype\\"><span class=\\"tabtype\\"><a data-category=\\"all\\" class=\\"cur\\">全部</a> |<a data-category=\\"china\\">国产片</a> |<a data-category=\\"other\\">引进片</a></span><a data-display=\\"list\\" class=\\"ico-display ico-list cur\\"><i></i></a><a data-display=\\"table\\" class=\\"ico-display ico-table\\"><i></i></a></div></div><div class=\\"boxofficelist\\"><div><dl><dt><em class=\\"paiming\\">排行</em><em class=\\"moviename\\">影片</em><em class=\\"pingfen\\">时光评分</em><em class=\\"piaofang\\">累计票房（元）</em></dt></dl><dd id=\\"MovieRankingHistory-rank-1\\"><div class=\\"movietopmod\\"><div class=\\"picbox\\" style=\\"background-color:#fbad68;\\"><i>01</i><a href=\\"http://movie.mtime.com/229733\\" target=\\"_blank\\" title=\\"战狼2/Wolf Warriors Ⅱ\\"><img src=\\"http://img5.mtime.cn/mt/2017/07/21/162643.51070350_185X2

In [15]:
type(MtimeRank.json())
MtimeRank.json().keys()

dict_keys(['expire', 'html', 'cache-hit'])

In [36]:
soup=bsp(MtimeRank.json()['html'])
soup

<html><body><div class="bocontent"><div><div class="typetitle"><h4>内地影史票房</h4><p class="updateday">更新时间：2019年2月11日</p><div class="showtype"><span class="tabtype"><a class="cur" data-category="all">全部</a> |<a data-category="china">国产片</a> |<a data-category="other">引进片</a></span><a class="ico-display ico-list cur" data-display="list"><i></i></a><a class="ico-display ico-table" data-display="table"><i></i></a></div></div><div class="boxofficelist"><div><dl><dt><em class="paiming">排行</em><em class="moviename">影片</em><em class="pingfen">时光评分</em><em class="piaofang">累计票房（元）</em></dt></dl><dd id="MovieRankingHistory-rank-1"><div class="movietopmod"><div class="picbox" style="background-color:#fbad68;"><i>01</i><a href="http://movie.mtime.com/229733" target="_blank" title="战狼2/Wolf Warriors Ⅱ"><img alt="战狼2/Wolf Warriors Ⅱ" height="190" src="http://img5.mtime.cn/mt/2017/07/21/162643.51070350_185X277X4.jpg" width="135"/></a></div><div class="txtbox"><h3><a href="http://movie.mtime.com/229733" 

In [263]:
## h3.a name in Chinese
## h4.a name in English
## test 
basic_info=soup.find_all("div",{"class":"txtbox"})
for info in basic_info:
    for i in info.contents:
        if i.name != 'b':
            if i.name == 'h3':
                print(type(i.a),type(i.contents),str(i.contents))
            else:
                print(i.name,type(i),type(i.text),i.text)
    print('')   
    for i in list(info.next_siblings):
        for j in i.contents:
            print(j.text)

<class 'bs4.element.Tag'> <class 'list'> [<a href="http://movie.mtime.com/229733" target="_blank">战狼2</a>]
h4 <class 'bs4.element.Tag'> <class 'str'> Wolf Warriors Ⅱ
p <class 'bs4.element.Tag'> <class 'str'> 首日1.02亿|首周9.97亿|连冠4周
p <class 'bs4.element.Tag'> <class 'str'> 2017年07月27日上映 2D/3D/IMAX
p <class 'bs4.element.Tag'> <class 'str'> 导演：吴京
p <class 'bs4.element.Tag'> <class 'str'> 主演：吴京/弗兰克·格里罗
p <class 'bs4.element.Tag'> <class 'str'> 发行公司：北京京西文化旅游股份有限公司...

7.4
16967人评分
56.82亿
累计人次：1.60亿
<class 'bs4.element.Tag'> <class 'list'> [<a href="http://movie.mtime.com/240425" target="_blank">红海行动</a>]
h4 <class 'bs4.element.Tag'> <class 'str'> Operation Red Sea
p <class 'bs4.element.Tag'> <class 'str'> 首日1.29亿|首周4.65亿
p <class 'bs4.element.Tag'> <class 'str'> 2018年02月16日上映 2D/3D/IMAX3D
p <class 'bs4.element.Tag'> <class 'str'> 导演：林超贤
p <class 'bs4.element.Tag'> <class 'str'> 主演：张译/黄景瑜
p <class 'bs4.element.Tag'> <class 'str'> 发行公司：博纳影业...

8.1
10110人评分
36.51亿
累计人次：9291.6万
<class 'bs4.eleme

In [322]:
## strin data
## some movie misses distribution companies
with open('mtime_data.txt', mode='w') as file:
    rank=1
    for page in range(10):
        parameters={"area":"china",
                   "type":"MovieRankingHistory",
                   "category":"all",
                   "page":f'{page}',
                   "display":"list",
                   "timestamp":f'{time.time()}',
                   "version":"07bb781100018dd58eafc3b35d42686804c6df8d",
                   "dataType":"json"}
        MtimeRank = rq.get("http://movie.mtime.com/boxoffice/",params=parameters) 
        if MtimeRank.status_code==200:
            print(f'Table {page+1} of the MTime Boxoffice successfully connected.')

        soup=bsp(MtimeRank.json()['html'])
        basic_info=soup.find_all("div",{"class":"txtbox"})
        
        for info in basic_info:
            for i in info.contents:
                if i.name !="b":
                    if i.name == 'h3':
                        file.write('排名: {}\n'.format(rank))
                        
                        id_name=str(i.contents)
                        file.write('{}\n'.format(re.search("http.+[0-9]+\"",id_name).group(0)[:-1]))
                        file.write('{}\n'.format(re.search("\>.+\<",id_name).group(0)[1:-1]))
                        
                    else:
                        file.write('{}\n'.format(i.text))
                        
            for i in list(info.next_siblings):
                for j in i.contents:
                    file.write('{}\n'.format(j.text))
            
            rank+=1
            

    file.flush()

Table 1 of the MTime Boxoffice successfully connected.
Table 2 of the MTime Boxoffice successfully connected.
Table 3 of the MTime Boxoffice successfully connected.
Table 4 of the MTime Boxoffice successfully connected.
Table 5 of the MTime Boxoffice successfully connected.
Table 6 of the MTime Boxoffice successfully connected.
Table 7 of the MTime Boxoffice successfully connected.
Table 8 of the MTime Boxoffice successfully connected.
Table 9 of the MTime Boxoffice successfully connected.
Table 10 of the MTime Boxoffice successfully connected.


In [130]:
## create data in a pandas data frame
id_link=[]
chinese_name=[]
english_name=[]

[<strong>1.02</strong>, <strong>9.97</strong>] 

[<strong>1.29</strong>, <strong>4.65</strong>] 

[<strong>3.41</strong>, <strong>9.90</strong>] 

[<strong>2.72</strong>, <strong>18.18</strong>] 

[<strong>1.60</strong>, <strong>11.72</strong>] 

[<strong>4.21</strong>, <strong>13.15</strong>] 

[<strong>2.27</strong>, <strong>9.02</strong>] 

[<strong>1.63</strong>, <strong>6.65</strong>] 

[<strong>3.46</strong>, <strong>3.46</strong>] 

[<strong>3.87</strong>, <strong>12.13</strong>] 



In [313]:
## test
test='[<a href="http://movie.mtime.com/229733" target="_blank">战狼2</a>]'
print(test)
print(re.search("http.+[0-9]+\"",test).group(0)[:-1])
print(re.search("\>.+\<",test).group(0)[1:-1])

[<a href="http://movie.mtime.com/229733" target="_blank">战狼2</a>]
http://movie.mtime.com/229733
战狼2


In [315]:
'发行' in "公司"

False